In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np

%matplotlib inline
import os
import pandas as pd
import numpy as np
import xgboost
import matplotlib.pyplot as plt
import scikitplot as sckplt
import time
import re
import scipy.stats as stat
from math import sqrt
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn import linear_model, tree, neighbors, svm, ensemble

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [2]:
# setting OS directory
os.chdir('C:\\Users\\rckar\\OneDrive\\Documents\\MSBA\\Fall Semester\\6420 Predictive Analytics\\HW3')

In [3]:
# Reading Data
df = pd.read_excel("HW3.xlsx")
df.head(2)
# df.dtypes

,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
0,1,1,0,0,1,0,0,0,0,0,...,0,0,2,3662,3662,1,0,1,1,127.87
1,2,1,0,0,0,0,1,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0.00


In [4]:
# Data pre-processing

# checking for null values
df.isnull().values.any()

False

In [5]:
df_purchase = df[df['Purchase'] == 1]

# Selecting required columns
X_df = df_purchase.iloc[:,1:23]
y_df = df_purchase.iloc[:,24:]

##### Creating the Train and Test split

In [6]:
X_train, X_test_holdout, y_train, y_test_holdout = train_test_split(X_df, y_df, test_size=0.25, random_state=42)

##### Normalizing data

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled)

x_test_scaled = scaler.fit_transform(X_test_holdout)
X_test_holdout = pd.DataFrame(x_test_scaled)

### Linear Regression

In [8]:
start = time.process_time()
# create linear regression object 
lr = linear_model.LinearRegression()

# cross validation 
scores = cross_val_score(lr, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")

print("Below are the mean absolute errors for each model run")
print(scores)
print(" ")
print("Mean score: %0.2f " % (abs(scores.mean())))

# Model fit on training data and predicting on testing data
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test_holdout)

# Model performance on testing data
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
# print('Explained Variance:',metrics.explained_variance_score(y_test_holdout, y_pred))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))

print(" ")
print('Time Taken = ', time.process_time() - start)

Below are the mean absolute errors for each model run
[ -96.53684897  -82.46407438 -111.29601696 -104.23354129 -114.72630347]
 
Mean score: 101.85 
 
 
Model performance on testing data
Mean Absolute Error: 107.59440489615713
Root Mean Squared Error: 179.3309695816787
r2: 0.5135462735363512
 
Time Taken =  0.328125


## Lasso Regression

In [9]:
start = time.process_time()

lasso = linear_model.Lasso()

#Hyper Parameter tuning
param_set ={'alpha': list(np.arange(0.1,2,0.2))}
grid_lasso = GridSearchCV(lasso, param_grid = param_set, cv=5, scoring='neg_mean_absolute_error', verbose = 0)
grid_lasso.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_lasso.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_lasso.best_params_)

print(" ")
print("Best estimator")
print (grid_lasso.best_estimator_)

# predicting on test data
lasso = grid_lasso.best_estimator_
lasso.fit(X_train,y_train)
y_pred = lasso.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  98.27231057485693
 
Best parameters
{'alpha': 1.9000000000000004}
 
Best estimator
Lasso(alpha=1.9000000000000004, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
 
 
Model performance on testing data
Mean Absolute Error: 106.75669504641262
Root Mean Squared Error: 183.88506395287703
r2: 0.4885256586200427
Time Taken =  1.0625


## Ridge Regression

In [10]:
start = time.process_time()

ridge = linear_model.Ridge()

#Hyper Parameter tuning
param_set ={'alpha': list(np.arange(0.1,2,0.2))}
grid_ridge = GridSearchCV(ridge, param_grid = param_set, cv=5, scoring='neg_mean_absolute_error', verbose = 0)
grid_ridge.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_ridge.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_ridge.best_params_)

print(" ")
print("Best estimator")
print (grid_ridge.best_estimator_)

# predicting on test data
ridge = grid_ridge.best_estimator_
ridge.fit(X_train,y_train)
y_pred = ridge.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  100.50414928984068
 
Best parameters
{'alpha': 1.3000000000000003}
 
Best estimator
Ridge(alpha=1.3000000000000003, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='auto',
   tol=0.001)
 
 
Model performance on testing data
Mean Absolute Error: 107.74492892655539
Root Mean Squared Error: 182.0902445429324
r2: 0.49846147334580204
Time Taken =  2.3125


## KNN

In [11]:
start = time.process_time()

knn = neighbors.KNeighborsRegressor()

#Hyper Parameter tuning
param_set ={'n_neighbors': list(range(1,30)), 'weights': ["uniform", "distance"]}
grid_knn = GridSearchCV(knn, param_grid = param_set, cv=5, scoring='neg_mean_absolute_error', verbose = 0)
grid_knn.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_knn.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_knn.best_params_)

print(" ")
print("Best estimator")
print (grid_knn.best_estimator_)

# predicting on test data
knn = grid_knn.best_estimator_
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  111.09508560440752
 
Best parameters
{'n_neighbors': 26, 'weights': 'distance'}
 
Best estimator
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=26, p=2,
          weights='distance')
 
 
Model performance on testing data
Mean Absolute Error: 128.92246215381246
Root Mean Squared Error: 239.6096409308345
r2: 0.13156060090654254
Time Taken =  12.953125


## Decision Tree

In [12]:
start = time.process_time()

DTree = tree.DecisionTreeRegressor()

#Hyper parameter tuning
param_set ={'max_depth': range(1,20), 'min_samples_split' : range(2,30)}
grid_DTree = GridSearchCV(DTree, param_grid = param_set, cv=5, scoring='neg_mean_absolute_error')
grid_DTree.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_DTree.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_DTree.best_params_)

print(" ")
print("Best estimator")
print (grid_DTree.best_estimator_)

# predicting on test data
DTree = grid_DTree.best_estimator_
DTree.fit(X_train,y_train)
y_pred = DTree.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.r2_score(y_test_holdout, y_pred))

print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  98.67343434617351
 
Best parameters
{'max_depth': 4, 'min_samples_split': 11}
 
Best estimator
DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=11, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
 
 
Model performance on testing data
Mean Absolute Error: 108.29010017865403
Root Mean Squared Error: 198.91353890225153
r2: 0.40150623602769187
Time Taken =  32.0625


## Support Vector Regression

In [13]:
start = time.process_time()

SVR = svm.SVR()
# Hyper parameter tuning using GridSearch
param_set = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1,2,10,100,1000], 'epsilon':[0.05,0.1,0.2,0.3,0.5]},
                    {'kernel': ['linear'], 'C': [1,2,5,10,100], 'epsilon':[0.05,0.1,0.2,0.3,0.5]}]
grid_SVR = GridSearchCV(SVR, param_grid = param_set, cv=5, scoring='neg_mean_absolute_error')
grid_SVR.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_SVR.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_SVR.best_params_)

print(" ")
print("Best estimator")
print (grid_SVR.best_estimator_)

# predicting on test data
SVR = grid_SVR.best_estimator_
SVR.fit(X_train,y_train)
y_pred = SVR.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  96.65206568430278
 
Best parameters
{'C': 100, 'epsilon': 0.5, 'kernel': 'linear'}
 
Best estimator
SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.5,
  gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)
 
 
Model performance on testing data
Mean Absolute Error: 108.4197259572856
Root Mean Squared Error: 203.30354331313333
r2: 0.3939695377170195
Time Taken =  25.765625


## Ensemble model : Random Forest

In [14]:
start = time.process_time()
RF = ensemble.RandomForestRegressor()

# Hyper parameter tuning using GridSearch
param_set ={'max_depth': [3,10,20],
            'min_samples_split' :[4,5,10],
            'n_estimators': [100,250,500],
            'bootstrap':[True, False] ,
            'max_features':['auto','sqrt'],
            'n_jobs':[-1]
           }
grid_RF = GridSearchCV(RF, param_grid = param_set, cv=5, scoring='neg_mean_absolute_error')
grid_RF.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_RF.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_RF.best_params_)

print(" ")
print("Best estimator")
print (grid_RF.best_estimator_)

# predicting on test data
RF = grid_RF.best_estimator_
RF.fit(X_train,y_train)
y_pred = RF.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))
print(" ")
print('Time Taken = ', time.process_time() - start)

Best score achieved across all parameters:  94.91881209672553
 
Best parameters
{'bootstrap': True, 'max_depth': 3, 'max_features': 'auto', 'min_samples_split': 5, 'n_estimators': 500, 'n_jobs': -1}
 
Best estimator
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
 
 
Model performance on testing data
Mean Absolute Error: 101.76641456829184
Root Mean Squared Error: 181.0008807774893
r2: 0.5048482374926706
 
Time Taken =  362.21875


## Ensemble model : XGBoost

In [15]:
start = time.process_time()
XGB = xgboost.XGBRegressor(nthreads=-1)

a = st.beta(10, 1)

# Hyper parameter tuning using GridSearch
param_set ={"n_estimators": st.randint(3, 40),
            "max_depth": st.randint(3, 40),
            "learning_rate": st.uniform(0.05, 0.4),
            "colsample_bytree": a,
            "subsample": a,
            "gamma": st.uniform(0, 10),
            'n_jobs':[-1]
           }

grid_XGB = RandomizedSearchCV(XGB, param_set, cv=5, scoring='neg_mean_absolute_error')
grid_XGB.fit(X_train,y_train)

# examine the best model
print("Best score achieved across all parameters: ", abs(grid_XGB.best_score_))

# Dictionary containing the parameters used to generate that score
print(" ")
print("Best parameters")
print (grid_XGB.best_params_)

print(" ")
print("Best estimator")
print (grid_XGB.best_estimator_)

# predicting on test data
XGB = grid_XGB.best_estimator_
XGB.fit(X_train,y_train)
y_pred = XGB.predict(X_test_holdout)

# print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
# print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
# print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))
# print(" ")
print('Time Taken = ', time.process_time() - start)

[00:37:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:37:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:37:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:37:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:37:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:37:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:37:34] WARNI

[00:37:39] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best score achieved across all parameters:  96.13404741170245
 
Best parameters
{'colsample_bytree': 0.8594916060785803, 'gamma': 9.465277258470813, 'learning_rate': 0.2134110737570123, 'max_depth': 4, 'n_estimators': 34, 'n_jobs': -1, 'subsample': 0.9595729552178727}
 
Best estimator
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8594916060785803,
       gamma=9.465277258470813, importance_type='gain',
       learning_rate=0.2134110737570123, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=34, n_jobs=-1,
       nthread=None, nthreads=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.9595729552178727, verbosity=1)
[00:37:39] WARNING:

In [16]:
# predicting on test data
XGB = grid_XGB.best_estimator_
XGB.fit(X_train,y_train)
y_pred = XGB.predict(X_test_holdout)

print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', sqrt(metrics.mean_squared_error(y_test_holdout, y_pred)))
print('r2:',metrics.explained_variance_score(y_test_holdout, y_pred))

[00:37:40] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
 
 
Model performance on testing data
Mean Absolute Error: 106.70936235107423
Root Mean Squared Error: 191.04454003178867
r2: 0.4479237811179464


## Neural Network

##### Shallow Network

In [17]:
def build_model():
    model = Sequential()
    model.add(Dense(units=44, activation = 'relu', input_dim=22))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_absolute_error',  metrics=['mae','mse'])
    return model

start = time.process_time()

NN_Regressor = KerasRegressor(build_fn=build_model, batch_size=40,epochs=100)    

scores = cross_val_score(NN_Regressor, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")

NN = NN_Regressor.fit(X_train,y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
600/600 [==============================] - 0s 387us/step - loss: 201.5887 - mae: 201.5887 - mse: 83867.1172
Epoch 2/100
600/600 [==============================] - 0s 55us/step - loss: 201.0253 - mae: 201.0253 - mse: 83639.7188
Epoch 3/100
600/600 [==============================] - 0s 46us/step - loss: 200.3855 - mae: 200.3855 - mse: 83386.5234
Epoch 4/100
600/600 [==============================] - 0s 50us/step - loss: 199.5787 - mae: 199.5787 - mse: 83072.1875
Epoch 5/100
600/600 [==============================] - 0s 47us/step - loss: 198.5416 - mae: 198.5417 - mse: 82667.2734
Epoch 6/100
600/600 [==============================] - 0s 50us/step - loss: 197.2402 - mae: 197.2402 - mse: 82139.5156
Epoch 7/100
600/600 [==============================] - 0s 47us/step - loss: 195.6639 - mae: 195.6638 - mse: 81504.1797
Epoch 8/100
600/600 [=========================

600/600 [==============================] - 0s 48us/step - loss: 118.5970 - mae: 118.5970 - mse: 49274.3320
Epoch 33/100
600/600 [==============================] - 0s 43us/step - loss: 117.9960 - mae: 117.9960 - mse: 48790.7227
Epoch 34/100
600/600 [==============================] - 0s 48us/step - loss: 117.5783 - mae: 117.5783 - mse: 48434.7266
Epoch 35/100
600/600 [==============================] - 0s 48us/step - loss: 117.2921 - mae: 117.2921 - mse: 48138.2500
Epoch 36/100
600/600 [==============================] - 0s 45us/step - loss: 117.0759 - mae: 117.0759 - mse: 47897.6875
Epoch 37/100
600/600 [==============================] - 0s 47us/step - loss: 116.9036 - mae: 116.9036 - mse: 47697.1250
Epoch 38/100
600/600 [==============================] - 0s 48us/step - loss: 116.7456 - mae: 116.7456 - mse: 47503.7070
Epoch 39/100
600/600 [==============================] - 0s 48us/step - loss: 116.6104 - mae: 116.6104 - mse: 47339.3398
Epoch 40/100
600/600 [==============================]

600/600 [==============================] - 0s 49us/step - loss: 116.8396 - mae: 116.8396 - mse: 46752.2852
Epoch 68/100
600/600 [==============================] - 0s 50us/step - loss: 116.7360 - mae: 116.7360 - mse: 46739.8711
Epoch 69/100
600/600 [==============================] - 0s 52us/step - loss: 116.6309 - mae: 116.6309 - mse: 46714.6797
Epoch 70/100
600/600 [==============================] - 0s 45us/step - loss: 116.5366 - mae: 116.5366 - mse: 46648.1719
Epoch 71/100
600/600 [==============================] - 0s 43us/step - loss: 116.4413 - mae: 116.4413 - mse: 46588.9102
Epoch 72/100
600/600 [==============================] - 0s 47us/step - loss: 116.3606 - mae: 116.3606 - mse: 46501.1641
Epoch 73/100
600/600 [==============================] - 0s 47us/step - loss: 116.2654 - mae: 116.2654 - mse: 46462.9297
Epoch 74/100
600/600 [==============================] - 0s 45us/step - loss: 116.1694 - mae: 116.1694 - mse: 46410.3086
Epoch 75/100
600/600 [==============================]

600/600 [==============================] - 0s 52us/step - loss: 193.6333 - mae: 193.6333 - mse: 78081.7656
Epoch 3/100
600/600 [==============================] - 0s 52us/step - loss: 192.8992 - mae: 192.8992 - mse: 77799.1719
Epoch 4/100
600/600 [==============================] - 0s 52us/step - loss: 192.0066 - mae: 192.0066 - mse: 77457.8516
Epoch 5/100
600/600 [==============================] - 0s 46us/step - loss: 190.8761 - mae: 190.8761 - mse: 77039.8203
Epoch 6/100
600/600 [==============================] - 0s 52us/step - loss: 189.4893 - mae: 189.4894 - mse: 76506.8672
Epoch 7/100
600/600 [==============================] - 0s 52us/step - loss: 187.7790 - mae: 187.7790 - mse: 75864.9844
Epoch 8/100
600/600 [==============================] - 0s 45us/step - loss: 185.7518 - mae: 185.7517 - mse: 75096.5781
Epoch 9/100
600/600 [==============================] - 0s 47us/step - loss: 183.4865 - mae: 183.4865 - mse: 74230.0078
Epoch 10/100
600/600 [==============================] - 0s 5

750/750 [==============================] - 0s 48us/step - loss: 116.6073 - mae: 116.6073 - mse: 46676.5156
Epoch 39/100
750/750 [==============================] - 0s 51us/step - loss: 116.4824 - mae: 116.4824 - mse: 46552.0898
Epoch 40/100
750/750 [==============================] - 0s 47us/step - loss: 116.3619 - mae: 116.3619 - mse: 46483.5234
Epoch 41/100
750/750 [==============================] - 0s 48us/step - loss: 116.2503 - mae: 116.2503 - mse: 46388.6406
Epoch 42/100
750/750 [==============================] - 0s 44us/step - loss: 116.1407 - mae: 116.1407 - mse: 46321.7812
Epoch 43/100
750/750 [==============================] - 0s 49us/step - loss: 116.0448 - mae: 116.0447 - mse: 46219.2305
Epoch 44/100
750/750 [==============================] - 0s 49us/step - loss: 115.9298 - mae: 115.9298 - mse: 46161.5586
Epoch 45/100
750/750 [==============================] - 0s 49us/step - loss: 115.8360 - mae: 115.8360 - mse: 46111.4258
Epoch 46/100
750/750 [==============================]

In [18]:
print("Below are the scores for each model run")
print(scores)
print(" ")
print("Mean and variance: %0.2f (+/- %0.2f)" % (abs(scores.mean()), scores.std() * 2))

y_pred= NN_Regressor.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', metrics.mean_squared_error(y_test_holdout, y_pred))

print('Time Taken = ', time.process_time() - start)

Below are the scores for each model run
[-108.60291955 -115.56424163 -104.61176985 -109.91084491 -129.1727929 ]
 
Mean and variance: 113.57 (+/- 17.11)
 
 
Model performance on testing data
Mean Absolute Error: 132.336899041748
Root Mean Squared Error: 65480.48822352548
Time Taken =  54.1875


##### Medium Deep Network

In [19]:
def build_model():
    model = Sequential()
    model.add(Dense(units=22, activation = 'relu', input_dim=22))
    model.add(Dense(units=22, activation = 'relu'))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_absolute_error',  metrics=['mae','mse'])
    return model

start = time.process_time()

NN_Regressor = KerasRegressor(build_fn=build_model, batch_size=40,epochs=100)    

scores = cross_val_score(NN_Regressor, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")

NN = NN_Regressor.fit(X_train,y_train)

Epoch 1/100
600/600 [==============================] - 0s 521us/step - loss: 201.6505 - mae: 201.6505 - mse: 83891.0000
Epoch 2/100
600/600 [==============================] - 0s 52us/step - loss: 201.0971 - mae: 201.0971 - mse: 83666.3281
Epoch 3/100
600/600 [==============================] - 0s 55us/step - loss: 200.4282 - mae: 200.4281 - mse: 83385.2969
Epoch 4/100
600/600 [==============================] - 0s 60us/step - loss: 199.4010 - mae: 199.4011 - mse: 82998.7891
Epoch 5/100
600/600 [==============================] - 0s 55us/step - loss: 197.7650 - mae: 197.7650 - mse: 82341.0469
Epoch 6/100
600/600 [==============================] - 0s 58us/step - loss: 195.3042 - mae: 195.3042 - mse: 81345.4766
Epoch 7/100
600/600 [==============================] - 0s 50us/step - loss: 191.7153 - mae: 191.7153 - mse: 79950.7031
Epoch 8/100
600/600 [==============================] - 0s 58us/step - loss: 186.6175 - mae: 186.6175 - mse: 77933.9766
Epoch 9/100
600/600 [==========================

600/600 [==============================] - 0s 27us/step - loss: 111.9415 - mae: 111.9415 - mse: 44582.6875
Epoch 38/100
600/600 [==============================] - 0s 30us/step - loss: 111.7273 - mae: 111.7273 - mse: 44467.5117
Epoch 39/100
600/600 [==============================] - 0s 23us/step - loss: 111.5234 - mae: 111.5234 - mse: 44302.4688
Epoch 40/100
600/600 [==============================] - 0s 25us/step - loss: 111.3241 - mae: 111.3241 - mse: 44158.7461
Epoch 41/100
600/600 [==============================] - 0s 28us/step - loss: 111.1363 - mae: 111.1364 - mse: 43935.8516
Epoch 42/100
600/600 [==============================] - 0s 25us/step - loss: 110.8143 - mae: 110.8143 - mse: 43874.0820
Epoch 43/100
600/600 [==============================] - 0s 28us/step - loss: 110.6758 - mae: 110.6758 - mse: 44061.2109
Epoch 44/100
600/600 [==============================] - 0s 32us/step - loss: 110.5439 - mae: 110.5439 - mse: 43691.5820
Epoch 45/100
600/600 [==============================]

600/600 [==============================] - 0s 32us/step - loss: 103.8170 - mae: 103.8170 - mse: 38106.0781
Epoch 72/100
600/600 [==============================] - 0s 32us/step - loss: 103.4063 - mae: 103.4063 - mse: 37940.5469
Epoch 73/100
600/600 [==============================] - 0s 32us/step - loss: 103.0102 - mae: 103.0102 - mse: 37477.9414
Epoch 74/100
600/600 [==============================] - 0s 32us/step - loss: 102.4962 - mae: 102.4962 - mse: 37144.2617
Epoch 75/100
600/600 [==============================] - 0s 33us/step - loss: 102.0898 - mae: 102.0898 - mse: 36865.9766
Epoch 76/100
600/600 [==============================] - 0s 28us/step - loss: 101.7200 - mae: 101.7200 - mse: 36698.0352
Epoch 77/100
600/600 [==============================] - 0s 28us/step - loss: 101.2833 - mae: 101.2833 - mse: 36278.1172
Epoch 78/100
600/600 [==============================] - 0s 30us/step - loss: 100.9748 - mae: 100.9748 - mse: 35874.8867
Epoch 79/100
600/600 [==============================]

Epoch 39/100
600/600 [==============================] - 0s 32us/step - loss: 111.8486 - mae: 111.8486 - mse: 43872.4883
Epoch 40/100
600/600 [==============================] - 0s 38us/step - loss: 111.5172 - mae: 111.5172 - mse: 43691.0781
Epoch 41/100
600/600 [==============================] - 0s 33us/step - loss: 111.2397 - mae: 111.2397 - mse: 43570.2891
Epoch 42/100
600/600 [==============================] - 0s 28us/step - loss: 110.9916 - mae: 110.9916 - mse: 43329.1055
Epoch 43/100
600/600 [==============================] - 0s 30us/step - loss: 110.6945 - mae: 110.6945 - mse: 43199.7383
Epoch 44/100
600/600 [==============================] - 0s 35us/step - loss: 110.4241 - mae: 110.4241 - mse: 42974.0078
Epoch 45/100
600/600 [==============================] - 0s 32us/step - loss: 110.1657 - mae: 110.1657 - mse: 42941.8086
Epoch 46/100
600/600 [==============================] - 0s 35us/step - loss: 109.9327 - mae: 109.9327 - mse: 42588.8281
Epoch 47/100
600/600 [==================

600/600 [==============================] - 0s 40us/step - loss: 97.2893 - mae: 97.2893 - mse: 34022.6055
Epoch 76/100
600/600 [==============================] - 0s 42us/step - loss: 97.1236 - mae: 97.1236 - mse: 34158.4727
Epoch 77/100
600/600 [==============================] - 0s 35us/step - loss: 96.7320 - mae: 96.7320 - mse: 33739.9453
Epoch 78/100
600/600 [==============================] - 0s 40us/step - loss: 96.3702 - mae: 96.3702 - mse: 33377.7617
Epoch 79/100
600/600 [==============================] - 0s 42us/step - loss: 96.0802 - mae: 96.0802 - mse: 33405.1680
Epoch 80/100
600/600 [==============================] - 0s 38us/step - loss: 95.8304 - mae: 95.8304 - mse: 33298.0352
Epoch 81/100
600/600 [==============================] - 0s 41us/step - loss: 95.4836 - mae: 95.4836 - mse: 32925.1406
Epoch 82/100
600/600 [==============================] - 0s 38us/step - loss: 95.1917 - mae: 95.1917 - mse: 32692.9668
Epoch 83/100
600/600 [==============================] - 0s 40us/step 

In [20]:
print("Below are the scores for each model run")
print(scores)
print(" ")
print("Mean and variance: %0.2f (+/- %0.2f)" % (abs(scores.mean()), scores.std() * 2))

y_pred= NN_Regressor.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', metrics.mean_squared_error(y_test_holdout, y_pred))

print('Time Taken = ', time.process_time() - start)

Below are the scores for each model run
[ -94.6321493   -95.18351877  -97.39047255  -96.68357474 -112.04744902]
 
Mean and variance: 99.19 (+/- 13.01)
 
 
Model performance on testing data
Mean Absolute Error: 107.73620826843262
Root Mean Squared Error: 41054.84083833669
Time Taken =  38.5


##### Deep Network

In [21]:
def build_model():
    model = Sequential()
    model.add(Dense(units=11, activation = 'relu', input_dim=22))
    model.add(Dense(units=11, activation = 'relu'))
    model.add(Dense(units=11, activation = 'relu'))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_absolute_error',  metrics=['mae','mse'])
    return model

start = time.process_time()

NN_Regressor = KerasRegressor(build_fn=build_model, batch_size=40,epochs=100)    

scores = cross_val_score(NN_Regressor, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")

NN = NN_Regressor.fit(X_train,y_train)

Epoch 1/100
600/600 [==============================] - 0s 293us/step - loss: 201.5205 - mae: 201.5205 - mse: 83843.5156
Epoch 2/100
600/600 [==============================] - 0s 40us/step - loss: 201.1532 - mae: 201.1532 - mse: 83692.1328
Epoch 3/100
600/600 [==============================] - 0s 35us/step - loss: 200.6095 - mae: 200.6095 - mse: 83475.3281
Epoch 4/100
600/600 [==============================] - 0s 37us/step - loss: 199.7005 - mae: 199.7005 - mse: 83116.7031
Epoch 5/100
600/600 [==============================] - 0s 42us/step - loss: 197.9934 - mae: 197.9934 - mse: 82419.3281
Epoch 6/100
600/600 [==============================] - 0s 38us/step - loss: 194.6833 - mae: 194.6833 - mse: 81106.0625
Epoch 7/100
600/600 [==============================] - 0s 37us/step - loss: 188.5416 - mae: 188.5416 - mse: 78725.9141
Epoch 8/100
600/600 [==============================] - 0s 48us/step - loss: 177.8460 - mae: 177.8460 - mse: 74649.6562
Epoch 9/100
600/600 [==========================

Epoch 38/100
600/600 [==============================] - 0s 38us/step - loss: 108.3267 - mae: 108.3267 - mse: 42503.5977
Epoch 39/100
600/600 [==============================] - 0s 37us/step - loss: 107.9678 - mae: 107.9678 - mse: 42276.9688
Epoch 40/100
600/600 [==============================] - 0s 35us/step - loss: 107.6079 - mae: 107.6079 - mse: 42014.6250
Epoch 41/100
600/600 [==============================] - 0s 38us/step - loss: 107.2316 - mae: 107.2316 - mse: 42007.6914
Epoch 42/100
600/600 [==============================] - 0s 38us/step - loss: 106.8611 - mae: 106.8611 - mse: 41803.5820
Epoch 43/100
600/600 [==============================] - 0s 35us/step - loss: 106.4974 - mae: 106.4974 - mse: 41431.7812
Epoch 44/100
600/600 [==============================] - 0s 35us/step - loss: 106.2030 - mae: 106.2029 - mse: 41320.5430
Epoch 45/100
600/600 [==============================] - 0s 37us/step - loss: 105.8538 - mae: 105.8538 - mse: 40840.5586
Epoch 46/100
600/600 [==================

600/600 [==============================] - 0s 36us/step - loss: 89.2011 - mae: 89.2011 - mse: 25596.3145
Epoch 75/100
600/600 [==============================] - 0s 37us/step - loss: 89.0728 - mae: 89.0728 - mse: 25353.0195
Epoch 76/100
600/600 [==============================] - 0s 37us/step - loss: 89.0887 - mae: 89.0887 - mse: 25306.2793
Epoch 77/100
600/600 [==============================] - 0s 42us/step - loss: 88.8435 - mae: 88.8435 - mse: 25043.3730
Epoch 78/100
600/600 [==============================] - 0s 37us/step - loss: 88.8116 - mae: 88.8115 - mse: 25055.7852
Epoch 79/100
600/600 [==============================] - 0s 37us/step - loss: 88.8097 - mae: 88.8097 - mse: 25086.6074
Epoch 80/100
600/600 [==============================] - 0s 40us/step - loss: 88.3285 - mae: 88.3285 - mse: 24661.1465
Epoch 81/100
600/600 [==============================] - 0s 37us/step - loss: 88.2279 - mae: 88.2279 - mse: 24534.5410
Epoch 82/100
600/600 [==============================] - 0s 48us/step 

600/600 [==============================] - 0s 33us/step - loss: 114.1441 - mae: 114.1441 - mse: 44422.1484
Epoch 12/100
600/600 [==============================] - 0s 32us/step - loss: 113.6885 - mae: 113.6885 - mse: 44269.2383
Epoch 13/100
600/600 [==============================] - 0s 32us/step - loss: 113.3989 - mae: 113.3989 - mse: 44783.2383
Epoch 14/100
600/600 [==============================] - 0s 32us/step - loss: 112.9827 - mae: 112.9827 - mse: 44322.9648
Epoch 15/100
600/600 [==============================] - 0s 33us/step - loss: 112.6248 - mae: 112.6248 - mse: 43895.9961
Epoch 16/100
600/600 [==============================] - 0s 33us/step - loss: 112.2885 - mae: 112.2885 - mse: 43875.4336
Epoch 17/100
600/600 [==============================] - 0s 37us/step - loss: 111.9803 - mae: 111.9803 - mse: 43770.3398
Epoch 18/100
600/600 [==============================] - 0s 33us/step - loss: 111.6732 - mae: 111.6732 - mse: 43352.5234
Epoch 19/100
600/600 [==============================]

750/750 [==============================] - 0s 34us/step - loss: 99.3414 - mae: 99.3414 - mse: 35422.7617
Epoch 48/100
750/750 [==============================] - 0s 39us/step - loss: 98.7227 - mae: 98.7227 - mse: 34506.7539
Epoch 49/100
750/750 [==============================] - 0s 36us/step - loss: 98.2271 - mae: 98.2271 - mse: 34493.6836
Epoch 50/100
750/750 [==============================] - 0s 36us/step - loss: 97.6938 - mae: 97.6938 - mse: 33818.5859
Epoch 51/100
750/750 [==============================] - 0s 36us/step - loss: 97.1721 - mae: 97.1721 - mse: 33446.9141
Epoch 52/100
750/750 [==============================] - 0s 39us/step - loss: 96.7238 - mae: 96.7238 - mse: 32979.5039
Epoch 53/100
750/750 [==============================] - 0s 38us/step - loss: 96.0871 - mae: 96.0871 - mse: 32556.6191
Epoch 54/100
750/750 [==============================] - 0s 43us/step - loss: 95.7338 - mae: 95.7338 - mse: 32400.5195
Epoch 55/100
750/750 [==============================] - 0s 33us/step 

In [22]:
print("Below are the scores for each model run")
print(scores)
print(" ")
print("Mean and variance: %0.2f (+/- %0.2f)" % (abs(scores.mean()), scores.std() * 2))

y_pred= NN_Regressor.predict(X_test_holdout)
print(" ")
print(" ")
print('Model performance on testing data')

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_holdout, y_pred))  
print('Root Mean Squared Error:', metrics.mean_squared_error(y_test_holdout, y_pred))

print('Time Taken = ', time.process_time() - start)

Below are the scores for each model run
[ -91.19046319  -83.89188298  -99.1992263   -94.6201038  -107.01524451]
 
Mean and variance: 95.18 (+/- 15.49)
 
 
Model performance on testing data
Mean Absolute Error: 101.0555049255371
Root Mean Squared Error: 34979.84705879639
Time Taken =  41.4375
